In [1]:
from auramask.metrics import EmbeddingDistance, PerceptualSimilarity
from auramask.models import FaceEmbedEnum

2024-02-22 00:03:16.009605: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-22 00:03:16.047589: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-22 00:03:16.047613: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-22 00:03:16.048516: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-22 00:03:16.054847: I tensorflow/core/platform/cpu_feature_guar

Using TensorFlow backend


## Inputs

### Victim Models (F)

In [2]:
F = [FaceEmbedEnum.ARCFACE, FaceEmbedEnum.DEEPFACE, FaceEmbedEnum.DEEPID]
F_set = FaceEmbedEnum.build_F(F)

2024-02-22 00:03:19.121066: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-22 00:03:19.155700: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-22 00:03:19.157141: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

### HyperParams

In [3]:
lda = 0.2
# Backbones are 'vgg', 'alex', 'squeeze'
backbone = "vgg"

# Metrics

## Perceptual Similarity

In [4]:
lpips = PerceptualSimilarity(backbone=backbone, spatial=False)

In [5]:
lpips.get_config()

{'name': 'Lpips', 'model': {}}

## Embedding Distance

In [6]:
embeddist = EmbeddingDistance(F=F)

In [7]:
embeddist.get_config()

{'name': 'Embedding Distance',
 'Cosine Similarity': <keras.src.losses.CosineSimilarity at 0x7f4bc9269d90>,
 'F': ListWrapper([<FaceEmbedEnum.ARCFACE: 'ArcFace'>, <FaceEmbedEnum.DEEPFACE: 'DeepFace'>, <FaceEmbedEnum.DEEPID: 'DeepID'>])}

# Test
Testing the giving parts of the loss

In [8]:
import tensorflow_datasets as tfds
from keras_cv.layers import (
    Resizing,
    Rescaling,
    Augmenter,
    RandomContrast,
    RandomColorDegeneration,
)
from tensorflow.data import AUTOTUNE

BATCH_SIZE = 16

In [9]:
ds, info = tfds.load(
    "lfw",
    decoders=tfds.decode.PartialDecoding(
        {
            "image": True,
        }
    ),
    with_info=True,
    download=True,
    as_supervised=False,
)

In [10]:
train_ds = ds["train"]

In [11]:
SEED = int.from_bytes("Pochita Lives".encode())

augmenter = Augmenter(
    [
        Rescaling(1.0 / 255),
        Resizing(128, 128),
    ]
)

noise_addition = Augmenter(
    [
        RandomContrast([0.0, 1.0], [-0.5, 0.5], seed=SEED),
        RandomColorDegeneration([0, 0.5], seed=SEED),
    ]
)


def preprocess_data(images):
    inputs = {"images": images}
    outputs_1 = augmenter(inputs)
    outputs_2 = noise_addition(inputs)
    return outputs_1["images"], outputs_2["images"]

In [12]:
train_ds = (
    train_ds.batch(BATCH_SIZE)
    .map(lambda x: preprocess_data(x["image"]), num_parallel_calls=AUTOTUNE)
    .prefetch(AUTOTUNE)
)

## Metrics Computation

In [13]:
for batch in train_ds.take(5):
    embeddist.update_state(batch[0], batch[1])
    perceptual = lpips(batch[0], batch[1])
    print(
        "Metrics:\n",
        "\tF-Dist: %0.3f\n" % (embeddist.result()["FLoss"]),
        "\tPerceptual: %0.3f\n" % (perceptual),
    )
    embeddist.reset_state()

2024-02-22 00:03:41.036441: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
2024-02-22 00:03:41.095469: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-02-22 00:03:41.404216: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


{'FLoss': <tf.Variable 'FLoss:0' shape=() dtype=float32, numpy=0.81959945>, 'arcface': <tf.Variable 'FLoss_arcface:0' shape=() dtype=float32, numpy=0.9021367>, 'deepface': <tf.Variable 'FLoss_deepface:0' shape=() dtype=float32, numpy=0.67260313>, 'deepid': <tf.Variable 'FLoss_deepid:0' shape=() dtype=float32, numpy=0.88405865>}
<tf.Variable 'count:0' shape=() dtype=float32, numpy=1.0>
Metrics:
 	F-Dist: 0.820
 	Perceptual: 0.567

{'FLoss': <tf.Variable 'FLoss:0' shape=() dtype=float32, numpy=0.83037776>, 'arcface': <tf.Variable 'FLoss_arcface:0' shape=() dtype=float32, numpy=0.9368875>, 'deepface': <tf.Variable 'FLoss_deepface:0' shape=() dtype=float32, numpy=0.67565036>, 'deepid': <tf.Variable 'FLoss_deepid:0' shape=() dtype=float32, numpy=0.87859523>}
<tf.Variable 'count:0' shape=() dtype=float32, numpy=1.0>
Metrics:
 	F-Dist: 0.830
 	Perceptual: 1.167

{'FLoss': <tf.Variable 'FLoss:0' shape=() dtype=float32, numpy=0.8285075>, 'arcface': <tf.Variable 'FLoss_arcface:0' shape=() dtype=

2024-02-22 00:03:56.869986: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


{'FLoss': <tf.Variable 'FLoss:0' shape=() dtype=float32, numpy=0.8297803>, 'arcface': <tf.Variable 'FLoss_arcface:0' shape=() dtype=float32, numpy=0.9140101>, 'deepface': <tf.Variable 'FLoss_deepface:0' shape=() dtype=float32, numpy=0.6961373>, 'deepid': <tf.Variable 'FLoss_deepid:0' shape=() dtype=float32, numpy=0.87919337>}
<tf.Variable 'count:0' shape=() dtype=float32, numpy=1.0>
Metrics:
 	F-Dist: 0.830
 	Perceptual: 2.931

